Connect to Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks'

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


Import required modules:

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import torch

Set seed for PyTorch random number generation:

In [3]:
torch.random.manual_seed(0)

Open the original dataset and print its column:

In [4]:
#df = pd.read_csv("Datasets/new_dataset.csv", encoding="utf-16", sep="\t")
df = pd.read_csv("Datasets/claim_review.csv")
print(df.columns)

<ipython-input-4-8506a802bca8>:2: DtypeWarning: Columns (14,24,25,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Datasets/claim_review.csv")


Index(['id', '@context', '@type', 'claimReviewed', 'datePublished', 'url',
       'author.@type', 'author.name', 'author.url', 'itemReviewed.@type',
       'itemReviewed.author.name', 'itemReviewed.author.@type',
       'itemReviewed.author.sameAs', 'reviewRating.@type',
       'reviewRating.ratingValue', 'reviewRating.alternateName',
       'author.image', 'itemReviewed.name', 'itemReviewed.datePublished',
       'itemReviewed.firstAppearance.url', 'itemReviewed.firstAppearance.type',
       'itemReviewed.author.image', 'reviewRating.ratingExplanation',
       'itemReviewed.author.jobTitle', 'reviewRating.bestRating',
       'reviewRating.worstRating', 'reviewRating.image',
       'itemReviewed.appearance.1.url', 'itemReviewed.appearance.1.@type',
       'itemReviewed.appearance.2.url', 'itemReviewed.appearance.2.@type',
       'itemReviewed.appearance.3.url', 'itemReviewed.appearance.3.@type',
       'itemReviewed.appearance.4.url', 'itemReviewed.appearance.4.@type',
       'itemRevi

Load the model:

In [5]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    trust_remote_code=True,
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name, device = "cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setup input arguments for the LLM>

In [6]:
system_prompt = """You are an assistant that helps fact-checking claims.
  Given a claim and the label telling its veracity, you need to normalize the label to a standart format,
  between this choices: True, Mostly True, Mixture, Mostly False or False. Answer only with the label.
  For example:
  Claim: The Eiffel Tower is located in Germany
  Label: Totally False
  """

pipe = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

generation_args = {
  "max_new_tokens": 4,
  "return_full_text": False,
  "do_sample": False,
}

Map labels:

In [ ]:
batch_size = 4
batch_messages = []

for i in range(25, 49, batch_size):
  batch_messages.clear()
  for j in range(i, min(i + batch_size, 49)):
    claim = df["claimReviewed"].iloc[j]
    label = df["reviewRating.alternateName"].iloc[j]

    claim_review = f"Claim: {claim}\n Label: {label}"

    message = [
        {"role": "user", "content": system_prompt},
        {"role": "assistant", "content": "False"},
        {"role": "user", "content": "Claim: One plus one equals two\n Label: True"},
        {"role": "assistant", "content": "True"},
        {"role": "user", "content": claim_review},
    ]

    batch_messages.append(message)

    output = pipe(message, **generation_args)

  for k in range(len(batch_messages)):
      claim_review = batch_messages[k][-1]["content"]
      print(f"Claim_review: {claim_review}")
      print("Output of the model is: " + output[k]['generated_text'])